In [1]:
import numpy as np
import tvm
from tvm import te
from matplotlib import pyplot as plt

[11:41:01] /home/aditya/Downloads/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[11:41:01] /home/aditya/Downloads/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[11:41:01] /home/aditya/Downloads/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


## Simple MatMul

In [ ]:
M = 1024
K = 1024
N = 1024

A = te.placeholder((M,K),name='A')
B = te.placeholder((K,N),name='B')

k = te.reduce_axis((0,K), name = 'k')

comp = te.compute((M,N), lambda i, j: te.sum(A[i,k] * B[k,j], axis= k), name = "C")


## Default Schedule without any optimizations

In [10]:

# Create the new TensorIR schedule
te_schedule = te.create_schedule(comp.op)

# tile_x, tile_y = 32,32

# x, y = te_schedule[comp].op.axis
# xo,xi = te_schedule[comp].split(x,factor = tile_x)
# yo,yi = te_schedule[comp].split(y,factor = tile_y)

# te_schedule[comp].reorder(xo,yo,xi,yi)

# Lower TE schedule into TIR (creates a PrimFunc/IRModule)
prim_func = tvm.lower(te_schedule, [A, B, comp], name="main")

# Now create a TensorIR schedule
tir_schedule = tvm.tir.Schedule(prim_func)

# Print the lowered TIR
print(tir_schedule.mod.script())


# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 1024), "float32"), B: T.Buffer((1024, 1024), "float32"), C: T.Buffer((1024, 1024), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for i, j in T.grid(1024, 1024):
            C_1 = T.Buffer((1048576,), data=C.data)
            C_1[i * 1024 + j] = T.float32(0.0)
            for k in range(1024):
                cse_var_2: T.int32 = i * 1024
                cse_var_1: T.int32 = cse_var_2 + j
                A_1 = T.Buffer((1048576,), data=A.data)
                B_1 = T.Buffer((1048576,), data=B.data)
                C_1[cse_var_1] = C_1[cse_var_1] + A_1[cse_var_2 + k] * B_1[k * 1024 + j]


In [ ]:
default_func = tvm.build(te_schedule, [A, B, comp], target='llvm -mcpu=core-avx2', name='conv')
# timer = default_func.time_evaluator(default_func.entry_name, tvm.cpu(0), min_repeat_ms=100)

# data arrays for inputs and outputs to the function
a = tvm.nd.array(np.random.rand(M, K).astype("float32"))
b = tvm.nd.array(np.random.rand(K, N).astype("float32"))
c = tvm.nd.array(np.zeros((M,N),dtype = "float32"))

default_func(a,b,c)

# time the execution of the function
# res_default = timer(a, b, c)
# print("MatMul with default schedule finished in:", res_default.mean, "seconds")
     

None


e.g. conv2d,
conv2d_transpose, group_conv2d, fc)